In [305]:
import sys
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import cvxpy as cp
from collections import defaultdict

# Add custom paths
sys.path.insert(0, 'E:\\User\\Stevens\\MyRepo\\fold-opt-package\\fold_opt')

# Suppress warnings
warnings.filterwarnings("ignore")

from GMRES import *
from fold_opt import *

# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

from src.utils.myOptimization import (
    AlphaFairnesstorch,
    solveIndProblem, solve_closed_form, solve_coupled_group_alpha, solve_coupled_group_grad,
    compute_coupled_group_obj
)
from src.utils.myPrediction import generate_random_features, customPredictionModel
from src.utils.plots import visLearningCurve
from src.fairness.cal_fair_penalty import atkinson_loss, mean_abs_dev
from src.utils.features import get_all_features


Using device: cuda


In [306]:
def compute_coupled_group_obj_torch(d, b, group_idx, alpha, beta=None):
    """
    Calculates the objective value for the new alpha=beta formulation using torch tensors.
    """
    # Ensure all inputs are torch tensors
    if not torch.is_tensor(d):
        d = torch.tensor(d, dtype=torch.float32)
    if not torch.is_tensor(b):
        b = torch.tensor(b, dtype=torch.float32)
    if not torch.is_tensor(group_idx):
        group_idx = torch.tensor(group_idx, dtype=torch.float32)

    d = d.reshape(-1)
    b = b.reshape(-1)
    group_idx = group_idx.reshape(-1)

    epsilon = 1e-12
    y = b * d + epsilon
    unique_groups = torch.unique(group_idx)
    g_k_values = torch.zeros(len(unique_groups), dtype=torch.float32, device=y.device)

    for i, k in enumerate(unique_groups):
        members_mask = (group_idx == k)
        y_k = y[members_mask]
        if 0 < alpha < 1:
            g_k_values[i] = torch.sum(y_k ** (1 - alpha)) / (1 - alpha)
        elif alpha > 1:
            g_k_values[i] = (alpha - 1) / torch.sum(y_k ** (1 - alpha))
        elif abs(alpha - 1.0) < epsilon:
            g_k_values[i] = torch.sum(torch.log(y_k))
        else:  # alpha <= 0
            g_k_values[i] = torch.sum(y_k)

    if alpha == float('inf') or str(alpha).lower() == 'inf':
        objective_value = torch.min(g_k_values)
    elif abs(alpha - 1.0) < epsilon:
        objective_value = torch.sum(torch.log(g_k_values + epsilon))
    elif abs(alpha - 0.0) < epsilon:
        objective_value = torch.sum(g_k_values)
    else:
        f_alpha_values = (g_k_values ** (1 - alpha)) / (1 - alpha)
        objective_value = torch.sum(f_alpha_values)

    return objective_value


## Define Alpha & Q

In [307]:
alpha, Q = 1.5, 1000

In [308]:
def to_numpy_1d(x):
    """Return a 1-D NumPy array; error if the length is not > 1."""
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    x = np.asarray(x).reshape(-1)
    assert x.ndim == 1, f"expected 1-D, got shape {x.shape}"
    return x


In [309]:
# df = pd.read_csv('/Users/dennis/Downloads/2024-fall/research/Fairness-Decision-Focused-Loss/Organized-FDFL/src/data/data.csv')

df = pd.read_csv('E:\\User\\Stevens\\MyRepo\\Organized-FDFL\\src\\data\\data.csv')

df = df.sample(n=2000, random_state=1)

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# ---------- basic 1-D helpers ----------
def as_1d(a, dtype=np.float32):
    a = np.asarray(a, dtype=dtype).reshape(-1)   # (N,)
    if a.ndim != 1:
        raise ValueError(f"expect 1-D, got {a.shape}")
    return a

# transform the features
scaler = StandardScaler()

# risk   = as_1d(df['risk_score_t']) * 100
risk = np.array(df['benefit']) * 100.0  # scale to [0,100]
risk = np.maximum(risk,1)         # or whatever the true column is
gainF  = np.ones_like(risk, dtype=np.float32)
cost   = as_1d(df['cost_t_capped']) * 10.0
cost   = np.maximum(cost, 1)              # keep strictly positive
race   = as_1d(df['race'])  # keep as int

feats  = scaler.fit_transform(df[get_all_features(df)]).astype(np.float32)   # (N,p)



In [310]:
class optDataset(Dataset):
    def __init__(self, optmodel, feats, risk, gainF, cost, race, alpha=alpha, Q=Q):
        # Store as numpy arrays for now
        self.feats = feats
        self.risk = risk
        self.gainF = gainF
        self.cost = cost
        self.race = race
        self.optmodel = optmodel

        # Call optmodel (expects numpy arrays)
        sol = self.optmodel(self.risk, self.cost, self.race, Q=Q, alpha=alpha)
        obj = compute_coupled_group_obj_torch(sol, self.risk, self.race, alpha=alpha)

        # Convert everything to torch tensors for storage
        self.feats = torch.from_numpy(self.feats).float()
        self.risk = torch.from_numpy(self.risk).float()
        self.gainF = torch.from_numpy(self.gainF).float()
        self.cost = torch.from_numpy(self.cost).float()
        self.race = torch.from_numpy(self.race).float()
        self.sol = torch.from_numpy(sol).float()
        self.obj = torch.tensor(obj).float()

    def __len__(self):
        return len(self.feats)

    # def __getitem__(self, idx):
    #     return self.feats, self.risk, self.gainF, self.cost, self.race, self.sol, self.obj

    def __getitem__(self, idx):
        
        return ( self.feats[idx],
                self.risk[idx],
                self.gainF[idx],
                self.cost[idx],
                self.race[idx],
                self.sol[idx],    # or store per-item solutions; see note ▼
                self.obj )  


## Prediction Model

In [311]:
class FairRiskPredictor(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.1):
        super().__init__()
        self.model = nn.Sequential(
            # First layer with batch normalization
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            # Output layer
            nn.Linear(64, 1),
            nn.Softplus()
        )
            
    def forward(self, x):
        return self.model(x).squeeze(-1)

# Load Data

In [312]:
# Setup training parameters

optmodel = solve_coupled_group_alpha

feats_np  = np.asarray(feats)              # 2-D OK
gainF_np  = to_numpy_1d(gainF)
risk_np   = to_numpy_1d(risk)
cost_np   = to_numpy_1d(cost)
race_np   = to_numpy_1d(df['race'].values)


# Perform train-test split
feats_train, feats_test, gainF_train, gainF_test, risk_train, risk_test, cost_train, cost_test, race_train, race_test = train_test_split(
    feats, gainF, risk, cost, df['race'].values, test_size=0.5, random_state=2
)

print(f"Train size: {feats_train.shape[0]}")
print(f"Test size: {feats_test.shape[0]}")

dataset_train = optDataset(optmodel, feats_train, risk_train, gainF_train, cost_train, race_train, alpha=alpha, Q=Q)
dataset_test = optDataset(optmodel, feats_test, risk_test, gainF_test, cost_test, race_test, alpha=alpha, Q=Q)

# Create dataloaders
dataloader_train = DataLoader(dataset_train, batch_size=len(dataset_train), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_train), shuffle=False)

predmodel = FairRiskPredictor(feats_train.shape[1])
predmodel.to(DEVICE)
# save the initial model
# torch.save(predmodel.state_dict(), 'initial_model.pth')
# load the initial model

# self.sol is (N,) – __getitem__ returns a scalar component;
# DataLoader stacks to (B,), which is exactly what the training loop expects.


Train size: 1000
Test size: 1000


FairRiskPredictor(
  (model): Sequential(
    (0): Linear(in_features=152, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=64, out_features=1, bias=True)
    (4): Softplus(beta=1, threshold=20)
  )
)

# Alpha Fair Obj

In [313]:
def alpha_fair_individual(u, alpha):
    if alpha == 1:
        return torch.log(u).sum(-1)
    elif alpha == 0:
        return u.sum(-1)
    elif alpha == 'inf':
        return u.min(-1).values
    return (u**(1-alpha)/(1-alpha)).sum(-1)

In [314]:
def solve_optimization(gainF, risk, cost, alpha, Q):
    gainF = gainF.detach().cpu().numpy() if isinstance(gainF, torch.Tensor) else gainF
    risk = risk.detach().cpu().numpy() if isinstance(risk, torch.Tensor) else risk
    cost = cost.detach().cpu().numpy() if isinstance(cost, torch.Tensor) else cost

    risk = risk.clip(0.001)
    gainF, risk, cost = gainF.flatten(), risk.flatten(), cost.flatten()
    d = cp.Variable(risk.shape, nonneg=True)

    if gainF.shape != risk.shape or risk.shape != cost.shape:
        raise ValueError("Dimensions of gainF, risk, and cost do not match")

    utils = cp.multiply(cp.multiply(gainF, risk), d)
    constraints = [d >= 0, cp.sum(cost * d) <= Q]

    if alpha == 'inf':
        t = cp.Variable()
        objective = cp.Maximize(t)
        constraints.append(utils >= t)
    elif alpha == 1:
        objective = cp.Maximize(cp.sum(cp.log(utils)))
    elif alpha == 0:
        objective = cp.Maximize(cp.sum(utils))
    else:
        objective = cp.Maximize(cp.sum(utils**(1-alpha)) / (1-alpha))

    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=False, warm_start=True, mosek_params={'MSK_IPAR_LOG': 1})

    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")

    optimal_decision = d.value
    optimal_value = alpha_fair_individual(optimal_decision * gainF * risk, alpha)

    return optimal_decision, optimal_value

# Analytical Projection

In [315]:
def proj_budget(x, cost, Q, max_iter=100):
    """
    x : (B,n)   or (n,)   –– internally promoted to (B,n)
    cost : (n,) positive
    Q : scalar or length‑B tensor
    """
    batched = x.dim() == 2
    if not batched:                       # (n,)  →  (1,n)
        x = x.unsqueeze(0)

    B, n = x.shape
    cost = cost.to(x)
    Q    = torch.as_tensor(Q, dtype=x.dtype, device=x.device).reshape(-1, 1)  # (B,1)

    d    = x.clamp(min=0.)                # enforce non‑neg
    viol = (d @ cost) > Q.squeeze(1)      # which rows violate the budget?

    if viol.any():
        dv, Qv = d[viol], Q[viol]
        lam_lo = torch.zeros_like(Qv.squeeze(1))
        lam_hi = (dv / cost).max(1).values   # upper bound for λ⋆

        for _ in range(max_iter):
            lam_mid = 0.5 * (lam_lo + lam_hi)
            trial   = (dv - lam_mid[:, None] * cost).clamp(min=0.)
            too_big = (trial @ cost) > Qv.squeeze(1)
            lam_lo[too_big] = lam_mid[too_big]
            lam_hi[~too_big]= lam_mid[~too_big]

        d[viol] = (dv - lam_hi[:, None] * cost).clamp(min=0.)

    return d if batched else d.squeeze(0)   # restore original rank


# Cvxpylayer Projection

In [316]:
from cvxpylayers.torch.cvxpylayer import CvxpyLayer
# --- 1. Define the Differentiable Projection Layer ---
def get_differentiable_projection(n, cost_np, Q_val):
    """
    Creates a differentiable projection layer using CvxpyLayer.
    This layer solves the projection problem:
        minimize   ||d - z||_2^2
        subject to cost^T * d <= Q
                   d >= 0
    """
    # Define CVXPY variables and parameters
    d_var = cp.Variable(n)
    z_param = cp.Parameter(n)
    objective = cp.Minimize(cp.sum_squares(d_var - z_param))
    constraints = [cost_np @ d_var <= Q_val, d_var >= 0]
    problem = cp.Problem(objective, constraints)
    return CvxpyLayer(problem, parameters=[z_param], variables=[d_var])


# Fold-Opt Layer

In [317]:
def pgd_step_cvxpylayer(r, d, g, race, cost, Q, alpha, lr, projection_layer, group=False):
    """
    Performs one PGD step using the CvxpyLayer for projection.
    """
    # Temporarily enable gradients for the internal gradient calculation
    with torch.enable_grad():
        # The input 'd' comes from the solver and has no grad history.
        # We clone it and require gradients to trace the PGD update.
        d_clone = d.clone().requires_grad_(True)
    
        if not group:
            obj = alpha_fair_individual(d_clone * r * g, alpha=alpha)
        else:
            obj = compute_coupled_group_obj_torch(d_clone, r, race, alpha=alpha)
        
        # Compute gradient of the objective w.r.t. the decision `d_clone`
        # This now happens within the `enable_grad` context.
        grad_d, = torch.autograd.grad(obj, d_clone, create_graph=True)

    # Perform the gradient ascent step
    unprojected_d = d + lr * grad_d
    
    # Project back to the feasible set using the differentiable layer
    projected_d, = projection_layer(unprojected_d)
    
    return projected_d

def make_foldopt_layer_cvxpylayer(g, cost, race, alpha, Q, group, lr=5e-3, n_fixedpt=50, backprop_rule='FPI'):
    """
    Factory function to create a FoldOptLayer with a differentiable PGD update.
    """
    # Detach tensors that are not parameters of the prediction model but are used in the optimization.
    # This prevents trying to compute gradients with respect to them.
    g_detached = g.detach()
    cost_detached = cost.detach()
    race_detached = race.detach()

    # --- Create the differentiable projection layer once ---
    n_vars = cost.shape[0]
    cost_np = cost_detached.cpu().numpy()
    Q_val = Q.item() if isinstance(Q, torch.Tensor) else Q
    projection_layer = get_differentiable_projection(n_vars, cost_np, Q_val)

    # --- Solver function (no gradients needed) ---
    def solver_fn(r_b): # r_b is batched, expected shape (B, n)
        # Assuming the solver can only handle one instance at a time.
        # If your solver is batched, you can simplify this.
        d_list = []
        for r_single in r_b:
            r_np = r_single.detach().cpu().numpy()
            cost_np_local = cost_detached.cpu().numpy()
            race_np_local = race_detached.cpu().numpy()
            
            if group:
                d_np = solve_coupled_group_alpha(r_np, cost_np_local, race_np_local, Q=Q_val, alpha=alpha)
            else:
                d_np,_ = solve_closed_form(g_detached.cpu().numpy(), r_np, cost_np_local, Q=Q_val, alpha=alpha)
                # d_np, _ = solve_optimization(g_detached.cpu().numpy(), r_np, cost_np, alpha, Q)

            
            d_list.append(torch.from_numpy(d_np).to(r_b.device, r_b.dtype))
        return torch.stack(d_list)


    # --- Differentiable update function (closure) ---
    def update_fn(r, d_star):
        # r and d_star are batched
        g_b = g_detached.expand_as(r)
        
        # Call the PGD step with the captured projection_layer and group flag
        return pgd_step_cvxpylayer(r, d_star, g_b, race_detached, cost_detached, Q, alpha, lr, projection_layer, group)

    return FoldOptLayer(solver_fn, update_fn, n_iter=n_fixedpt, backprop_rule=backprop_rule)

# Train Func

In [318]:
def train_model_differentiable_dfl(
    predmodel,
    feats_train, risk_train, gainF_train, cost_train, race_train,
    alpha, Q,
    group=True,
    num_epochs=50,
    lr_pred=1e-3,
    pgd_lr=5e-3,
    n_fixedpt=50,
    lambda_fairness=0.0,
    fairness_type='atkinson'
):
    device = next(predmodel.parameters()).device
    optimizer = torch.optim.Adam(predmodel.parameters(), lr=lr_pred)
    
    feats = torch.from_numpy(feats_train).to(device)
    risk = torch.from_numpy(risk_train).to(device)
    gainF = torch.from_numpy(gainF_train).to(device)
    cost = torch.from_numpy(cost_train).to(device)
    race = torch.from_numpy(race_train).to(device)

    # --- Pre-calculate optimal solution (ground truth) ---
    if group:
        opt_d_np = solve_coupled_group_alpha(risk.cpu().numpy(), cost.cpu().numpy(), race.cpu().numpy(), Q, alpha)
    else:
        opt_d_np,_ = solve_closed_form(gainF.cpu().numpy(), risk.cpu().numpy(), cost.cpu().numpy(), Q, alpha)
    opt_d = torch.from_numpy(opt_d_np).to(device)
    
    # --- CORRECTED: Calculate optimal objective ---
    # The function already aggregates over groups, so no loop is needed.
    if group:
        opt_obj = compute_coupled_group_obj_torch(opt_d, risk, race, alpha)
    else:
        opt_obj = alpha_fair_individual(opt_d * risk * gainF, alpha)

    # --- Build the FoldOpt layer with the 'group' flag ---
    differentiable_opt_layer = make_foldopt_layer_cvxpylayer(
        gainF, cost, race, alpha, Q, group, lr=pgd_lr, n_fixedpt=n_fixedpt
    )

    logs = {"loss": [], "regret": [], "fairness": []}
    
    for epoch in range(1, num_epochs + 1):
        predmodel.train()
        
        pred_risk = predmodel(feats).clamp(min=1)
        d_pred = differentiable_opt_layer(pred_risk.unsqueeze(0)).squeeze(0)
        
        # --- Loss Calculation ---
        if group:
            # Use d_pred here to keep the computation graph intact
            pred_obj = compute_coupled_group_obj_torch(d_pred, risk, race, alpha)
        else:
            pred_obj = alpha_fair_individual(d_pred * risk * gainF, alpha)

        regret = (opt_obj - pred_obj) / (torch.abs(opt_obj) + 1e-8)
        
        # --- Fairness Penalty ---
        fairness_penalty = torch.tensor(0.0, device=device)
        if lambda_fairness > 0:
            mode = 'between' if group else 'individual'
            if fairness_type == 'atkinson':
                fairness_penalty = atkinson_loss(pred_risk, risk, race, beta=0.5, mode=mode)
            elif fairness_type == 'mad':
                fairness_penalty = mean_abs_dev(pred_risk, risk, race, mode=mode)

        loss = regret + lambda_fairness * fairness_penalty
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(predmodel.parameters(), 1.0)
        optimizer.step()
        
        logs["loss"].append(loss.item())
        logs["regret"].append(regret.item())
        logs["fairness"].append(fairness_penalty.item())
            
    # --- Final Evaluation for Detailed Logging ---
    predmodel.eval()
    with torch.no_grad():
        final_pred_risk = predmodel(feats).clamp(min=1e-4)
        final_d_pred = differentiable_opt_layer(final_pred_risk.unsqueeze(0)).squeeze(0)
        
        eval_logs = {}
        unique_groups = torch.unique(race).cpu().numpy()
        for g in unique_groups:
            mask = (race == g)
            if mask.sum() == 0: continue
            eval_logs[f'G{int(g)}_mse'] = (final_pred_risk[mask] - risk[mask]).pow(2).mean().item()
            group_utility = (final_d_pred[mask] * risk[mask] * gainF[mask])
            eval_logs[f'G{int(g)}_decision_obj'] = alpha_fair_individual(group_utility, alpha).item()
            eval_logs[f'G{int(g)}_true_benefit'] = risk[mask].mean().item()

    return predmodel, logs, eval_logs

In [319]:
def run_dfl_experiment_grid(
    feats, risk, gainF, cost, race,
    alpha, Q,
    n_trials=3,
    base_seed=2025,
    group_settings=[True, False],
    lambda_grid=[0.0, 1.0],
    fairness_type_grid=['atkinson', 'mad'],
    num_epochs=50
):
    """
    Runs multiple random-split trials for each hyperparameter in the grid.
    """
    results_list = []
    
    for group in group_settings:
        for fairness_type in fairness_type_grid:
            for lam in lambda_grid:
                if lam == 0 and fairness_type != fairness_type_grid[0]:
                    continue

                trial_metrics = defaultdict(list)
                run_params = f"Group={group}, Lambda={lam}, Fairness={fairness_type}"
                print("\n" + "="*70)
                print(f"RUNNING EXPERIMENT: {run_params}")
                print("="*70)

                for t in range(n_trials):
                    print(f"--- Trial {t+1}/{n_trials} ---")
                    seed = base_seed + t
                    torch.manual_seed(seed)
                    np.random.seed(seed)
                    
                    (feats_tr, _, risk_tr, _, gainF_tr, _, cost_tr, _, race_tr, _) = train_test_split(
                        feats, risk, gainF, cost, race, test_size=0.5, random_state=seed
                    )
                    
                    predictor = FairRiskPredictor(input_dim=feats_tr.shape[1]).to(DEVICE)
                    
                    _, logs, eval_logs = train_model_differentiable_dfl(
                        predictor,
                        feats_tr, risk_tr, gainF_tr, cost_tr, race_tr,
                        alpha=alpha, Q=Q,
                        group=group,
                        lambda_fairness=lam,
                        fairness_type=fairness_type,
                        num_epochs=num_epochs
                    )
                    
                    trial_metrics['final_loss'].append(logs['loss'][-1])
                    trial_metrics['final_regret'].append(logs['regret'][-1])
                    trial_metrics['final_fairness_penalty'].append(logs['fairness'][-1])
                    for key, val in eval_logs.items():
                        trial_metrics[key].append(val)

                aggregated_row = {'Group': group, 'Lambda': lam, 'FairnessType': fairness_type}
                for key, values in trial_metrics.items():
                    aggregated_row[f'{key}_mean'] = np.mean(values)
                    aggregated_row[f'{key}_std'] = np.std(values)
                results_list.append(aggregated_row)

    return pd.DataFrame(results_list)

In [320]:
# --- 2. Set Parameters ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

ALPHA_GRID = [0.5, 0.8, 1.5, 2.0]
Q_BUDGET = 600

all_results = []
for alpha_val in ALPHA_GRID:
    print(f"\n{'='*30} Running for alpha={alpha_val} {'='*30}")
    results_df = run_dfl_experiment_grid(
        feats, risk, gainF, cost, race,
        alpha=alpha_val,
        Q=Q_BUDGET,
        n_trials=1,
        group_settings=[True],
        lambda_grid=[0.0, 1.0],
        fairness_type_grid=['atkinson', 'mad'],
        num_epochs=30
    )
    results_df['Alpha'] = alpha_val
    all_results.append(results_df)

# Concatenate all results
final_results_df = pd.concat(all_results, ignore_index=True)

# --- 4. Print Final Results Table ---
print("\n" + "="*90)
print(" " * 30 + "GRID SEARCH COMPLETE")
print("="*90)
# Reorder columns for better readability
hp_cols = ['Alpha', 'Group', 'Lambda', 'FairnessType']
metric_cols = sorted([c for c in final_results_df.columns if c not in hp_cols])
final_results_df = final_results_df[hp_cols + metric_cols]

with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None,
                       'display.width', 1200,
                       'display.float_format', '{:.4f}'.format):
    print(final_results_df)

Using device: cuda

============================== Running for alpha=0.5 ==============================

RUNNING EXPERIMENT: Group=True, Lambda=0.0, Fairness=atkinson
--- Trial 1/1 ---

RUNNING EXPERIMENT: Group=True, Lambda=1.0, Fairness=atkinson
--- Trial 1/1 ---

RUNNING EXPERIMENT: Group=True, Lambda=1.0, Fairness=mad
--- Trial 1/1 ---

============================== Running for alpha=0.8 ==============================

RUNNING EXPERIMENT: Group=True, Lambda=0.0, Fairness=atkinson
--- Trial 1/1 ---

RUNNING EXPERIMENT: Group=True, Lambda=1.0, Fairness=atkinson
--- Trial 1/1 ---

RUNNING EXPERIMENT: Group=True, Lambda=1.0, Fairness=mad
--- Trial 1/1 ---

============================== Running for alpha=1.5 ==============================

RUNNING EXPERIMENT: Group=True, Lambda=0.0, Fairness=atkinson
--- Trial 1/1 ---

RUNNING EXPERIMENT: Group=True, Lambda=1.0, Fairness=atkinson
--- Trial 1/1 ---

RUNNING EXPERIMENT: Group=True, Lambda=1.0, Fairness=mad
--- Trial 1/1 ---

===========

# Results

In [321]:
final_results_df[['Alpha', 'Group', 'Lambda', 'FairnessType', 'final_regret_mean']]

,Alpha,Group,Lambda,FairnessType,final_regret_mean
0,0.5,True,0.0,atkinson,0.074304
1,0.5,True,1.0,atkinson,0.076369
2,0.5,True,1.0,mad,0.078836
3,0.8,True,0.0,atkinson,0.001636
4,0.8,True,1.0,atkinson,0.001671
5,0.8,True,1.0,mad,0.001642
6,1.5,True,0.0,atkinson,0.027419
7,1.5,True,1.0,atkinson,0.028800
8,1.5,True,1.0,mad,0.030363
9,2.0,True,0.0,atkinson,0.402790


In [322]:
final_results_df.to_csv('fold-opt-n2000.csv', index=False)